In [5]:
! pip install bioservices

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 13.9 MB/s eta 0:00:00
  Created wheel for bioservices: filename=bioservices-1.11.2-py3-none-any.whl size=223228 sha256=d866af41337dde53dc6270c573ed8cacde18f4044b4c06a9d0267ecc4743b6ea
  Stored in directory: /root/.cache/pip/wheels/bf/ac/b3/dc05e53581bbb58641e9ac5242

In [6]:
import requests
from bioservices import QuickGO
import time

def gene_name_to_uniprot_id(gene_name: str) -> str:
    """
    Converts a gene name to the corresponding UniProt ID using a direct UniProt API request.

    Args:
        gene_name (str): The gene name or symbol (e.g., "BRCA1").

    Returns:
        str: The UniProt ID associated with the gene name, or None if not found.
    """
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        'query': f'gene_exact:{gene_name} AND organism_id:9606',  # Filtering for human (organism ID 9606)
        'fields': 'accession',  # Fetch only the UniProt accession ID
        'format': 'tsv',        # Requesting tab-separated values format
        'size': 1               # Limit results to one entry
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an error for HTTP error codes

        # Parse the response content
        lines = response.text.splitlines()
        if len(lines) > 1:  # First line is the header, next lines contain data
            uniprot_id = lines[1].split('\t')[0]  # Extract the UniProt ID from the first result
            return uniprot_id
        else:
            print(f"No UniProt ID found for gene: {gene_name}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching UniProt ID for {gene_name}: {e}")
        return None

Creating directory /root/.config/bioservices 


In [7]:
# Should print the UniProt ID for BRCA1, need Uniprot IDs for QuickGO API queries later
uniprot_id = gene_name_to_uniprot_id('BRCA1')
print(uniprot_id)

P38398


In [8]:
def get_gene_go_terms(gene: str) -> list:
    """
    Returns a list of GO Biological Process terms associated with a gene using QuickGO.

    Args:
        gene (str): The gene symbol or identifier (e.g., UniProt ID).

    Returns:
        list: A list of GO Biological Process terms (GO IDs) associated with the gene.
    """
    quickgo = QuickGO()

    try:
        # Correct aspect value for Biological Process is 'P'
        print(f"Querying GO terms for gene: {gene}")
        results = quickgo.Annotation(geneProductId=gene, geneProductType="protein", aspect="P")

        # Check if the results are in the expected format
        if isinstance(results, dict) and 'results' in results:
            # Extract GO terms from the results
            go_terms = [annotation['goId'] for annotation in results['results']
                        if annotation.get('goAspect') == 'biological_process']
            return go_terms
        else:
            print("Unexpected response format or empty response:", results)
            return []

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [9]:
# Output will be a list of GO biological process terms for the given gene's Uniprot ID
terms = get_gene_go_terms('P38398')
print(terms)

Creating directory /root/.cache/bioservices 
Welcome to Bioservices
It looks like you do not have a configuration file.
We are creating one with default values in /root/.config/bioservices/bioservices.cfg .
Done
Querying GO terms for gene: P38398
['GO:0051726', 'GO:0008630', 'GO:0044027', 'GO:0000724', 'GO:0000724', 'GO:0000724', 'GO:0006281', 'GO:0006281', 'GO:0006281', 'GO:0006282', 'GO:0006301', 'GO:0006302', 'GO:0006302', 'GO:0006302', 'GO:0006302', 'GO:0006302', 'GO:0006302', 'GO:0006310', 'GO:0006357', 'GO:0006357', 'GO:0006633', 'GO:0006974', 'GO:0006974', 'GO:0006974', 'GO:0007059', 'GO:0007095', 'GO:0007095', 'GO:0007095', 'GO:0007095', 'GO:0007098', 'GO:0007549', 'GO:0010212', 'GO:0010212', 'GO:0010212', 'GO:0010575', 'GO:0010628', 'GO:0016567', 'GO:0016567', 'GO:0030308', 'GO:0033147', 'GO:0035825', 'GO:0035825', 'GO:0043009', 'GO:0043009', 'GO:0044027', 'GO:0044818', 'GO:0044818', 'GO:0045717', 'GO:0045739', 'GO:0045739', 'GO:0045766', 'GO:0045786', 'GO:0045892', 'GO:004589

In [10]:
print(f'There are {len(terms)} GO BP terms associated with P38398/BRCA1')

There are 72 GO BP terms associated with P38398/BRCA1


In [11]:
# return a dictionary of all data associated with a given GO BP term

def fetch_go_term_details(term: str) -> dict:
    """
    Fetches and returns all available data for a single GO term using the QuickGO API.

    Args:
        term (str): A GO term ID (e.g., 'GO:0008150').

    Returns:
        dict: A dictionary containing the data for the GO term, or None if the request fails.
    """
    base_url = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms"
    headers = {'Accept': 'application/json'}

    retries = 3
    for attempt in range(retries):
        try:
            # detailed information related to the GO term
            response = requests.get(f"{base_url}/{term}", headers=headers)
            response.raise_for_status()  # Raise an exception for HTTP errors

            # full response data as a dictionary
            data = response.json()
            print(f"Full data for {term}:\n", data)
            return data

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed for {term}: {e}")
            time.sleep(2)  # wait two seconds

    # Return None if all attempts fail
    print(f"Failed to fetch data for {term} after {retries} attempts.")
    return None

# Go through all available GO BP terms in QuickGo, get gene counts

def fetch_all_go_bp_terms_and_counts() -> dict:
    """
    Fetches all GO Biological Process terms from QuickGO and retrieves their gene counts,
    printing the terms and counts as they are retrieved.

    Returns:
        dict: A dictionary with GO term IDs as keys and counts of associated genes as values.
    """
    base_url = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms"
    headers = {'Accept': 'application/json'}
    go_terms = []
    params = {
        'aspect': 'biological_process',  # restrict to Biological Process terms
        'pageSize': 1000,
        'page': 1
    }
    term_gene_counts = {}

    try:
        while True:
            response = requests.get(base_url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            # collect GO term IDs
            terms = [term['id'] for term in data.get('results', [])]
            go_terms.extend(terms)

            # print each retrieved term and get counts
            for term in terms:
                print(f"Retrieved GO term: {term}")
                # fetch the gene count for this term
                counts = get_term_gene_counts([term])  # fetch counts for the current term
                term_gene_counts.update(counts)  # update the main dictionary with fetched counts

            # use total number of terms if available
            total_terms = data.get('pageInfo', {}).get('total', 0)
            current_page = data.get('pageInfo', {}).get('current', 1)
            results_per_page = data.get('pageInfo', {}).get('resultsPerPage', len(data.get('results', [])))

            # do more pages exist?
            if current_page * results_per_page >= total_terms:
                break  # exit if the last page is reached

            params['page'] += 1  # increment the page number

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching GO BP terms: {e}")

    return term_gene_counts

# count genes associated with GO BP term, if server error print out all avaiable data - most likely term is 'obsolete', in that case print 'Obsolete? True'

def get_term_gene_counts(go_terms: list) -> dict:
    """
    Fetches the count of gene products (genes) associated with each GO term using QuickGO's annotation endpoint,
    with retry logic and error handling.

    Args:
        go_terms (list): A list of GO term IDs (e.g., ['GO:0008150', 'GO:0009987']).

    Returns:
        dict: A dictionary with GO term IDs as keys and counts of associated genes as values.
    """
    base_url = "https://www.ebi.ac.uk/QuickGO/services/annotation/search"
    headers = {'Accept': 'application/json'}
    term_gene_counts = {}

    for term in go_terms:
        retries = 3
        for attempt in range(retries):
            try:
                # query QuickGO for annotations related to the GO term
                params = {
                    'goId': term,
                    'limit': 0,  # Set limit to 0 because we only need the count
                    'facetField': 'geneProductId'
                }
                response = requests.get(base_url, headers=headers, params=params)
                response.raise_for_status()  # Raise an exception for HTTP errors

                data = response.json()

                # count of gene products (genes) from the response
                gene_count = data.get('numberOfHits', 0)

                # store in dictionary
                term_gene_counts[term] = gene_count
                print(f"Count for {term}: {gene_count}")
                break  # break if retry loop successful

            except requests.exceptions.RequestException as e:
                print(f"Attempt {attempt + 1} failed for {term}: {e}")
                time.sleep(2)  # wait a couple seconds

                # for last attempt, log the error and set the count to 0
                if attempt == retries - 1:
                    print(f"Failed to fetch data for {term} after {retries} attempts.")
                    term_gene_counts[term] = 0
                    details=fetch_go_term_details(term=term)
                    obs = details['results'][0]['isObsolete']
                    print(f'Obsolete term?: {obs}')
                    print(details)

    return term_gene_counts

In [12]:
# get counts for all GO BP terms found, if server returns error print out full data returned for that term from API query
term_gene_counts = fetch_all_go_bp_terms_and_counts()

Retrieved GO term: GO:0006600
Count for GO:0006600: 5835
Retrieved GO term: GO:0006601
Count for GO:0006601: 5732
Retrieved GO term: GO:0006611
Count for GO:0006611: 45363
Retrieved GO term: GO:0006612
Count for GO:0006612: 582930
Retrieved GO term: GO:0006610
Count for GO:0006610: 200
Retrieved GO term: GO:0006604
Count for GO:0006604: 0
Retrieved GO term: GO:0006605
Count for GO:0006605: 1308980
Retrieved GO term: GO:0006602
Count for GO:0006602: 874
Retrieved GO term: GO:0006603
Count for GO:0006603: 30850
Retrieved GO term: GO:0006608
Attempt 1 failed for GO:0006608: 500 Server Error:  for url: https://www.ebi.ac.uk/QuickGO/services/annotation/search?goId=GO%3A0006608&limit=0&facetField=geneProductId
Attempt 2 failed for GO:0006608: 500 Server Error:  for url: https://www.ebi.ac.uk/QuickGO/services/annotation/search?goId=GO%3A0006608&limit=0&facetField=geneProductId
Attempt 3 failed for GO:0006608: 500 Server Error:  for url: https://www.ebi.ac.uk/QuickGO/services/annotation/search

KeyboardInterrupt: 

In [13]:
def check_shared_go_terms(gene_1: str, gene_2: str, n_shared_threshold: int = None) -> int:
    """
    Check if two genes share a GO Biological Process term. Optionally filter the terms
    by a threshold of how many genes share the term.

    Args:
        gene_1 (str): The first gene to compare. UNIPROT ID, not gene name.
        gene_2 (str): The second gene to compare. UNIPROT ID, not gene name.
        n_shared_threshold (int, optional): Minimum number of genes that must share a term
                                            for it to be considered in the comparison.

    Returns:
        int: 1 if the genes share at least one filtered GO term, 0 otherwise.
    """
    # get_gene_go_terms returns a list, get ordered set for comparison
    go_terms_1 = set(get_gene_go_terms(gene_1))
    go_terms_2 = set(get_gene_go_terms(gene_2))

    # if threshold is none, just compare - do they share any GO terms? Then 1, else - 0.
    if n_shared_threshold is None:
        shared_terms = go_terms_1 & go_terms_2
        return 1 if shared_terms else 0

    # Get counts of genes for each GO term
    terms_1 = get_term_gene_counts(go_terms_1)
    terms_2 = get_term_gene_counts(go_terms_2)

    def filter_terms(term_dict, thresh):
      filt = {key: value for key, value in term_dict.items() if value >= thresh}
      return set(list(filt.keys()))

    shared_filtered_terms = filter_terms(terms_1, n_shared_threshold) & filter_terms(terms_2, n_shared_threshold)

    return 1 if shared_filtered_terms else 0

In [14]:
BRCA1 = "P38398"
TP53 = "P04637"

In [15]:
check_shared_go_terms(BRCA1, TP53)

Querying GO terms for gene: P38398
Querying GO terms for gene: P04637


1